In [320]:
# Análise foi realizada com a base de dados disponibilizada no link abaixo:
# http://ai.stanford.edu/~amaas/data/sentiment/


# Obs.: Como será feita apenas a validação dos dados que estão dentro das pastas:
# train/pos e train/neg
# test/pos e test/neg
# Deve remover a pasta "unsup" e os arquivos: labeledBow.feat, unsupBow.feat, urls_neg, urls_pos, urls_unsup
# tanto para a pasta train como para pasta test.


# ANTES DE EXECUTAR, LEIA O NOTEBOOK E MODIFIQUE O CAMINHO PARA VARIÁVEL "path"
# TANTO PARA A PASTA "pos" QUANTO "neg" 
# Exemplo: path = r'caminho da sua máquina onde encontra-se o arquivo\aclImdb\train\pos'

import pandas as pd
import glob 
import re
import unicodedata
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Carrega dados de treinamento e teste
from sklearn.datasets import load_files

# Grid Search : para ajuste de parâmetro de LogisticRegression. 
# Determina qual valor de coefficeint 'C' fornece melhor precisão.
from sklearn.model_selection import GridSearchCV

# Foi utilizado o técnica de "LogisticRegression", 
# pois foi verificado que é bastante usada para problemas de classificação binária 
# (problemas com dois valores de classe).
from sklearn.linear_model import LogisticRegression

# use o caminho onde a pasta "aclImdb" estiver no seu computar ex: ...\aclImdb\train\pos
path = r'C:\Users\alison.americo.lira\Documents\classificador_de_filmes\aclImdb\train\pos' 

# Caminho dos arquivos + sua extenção.
# Onde será feita iterações e leitura dos arquivos. 
all_files = glob.glob(path + "/*.txt")
li = []
for filename in all_files:
    df = pd.read_table(filename, index_col=None, header=None, sep="|")
    li.append(df)

In [321]:
# Irá fazer o merge no frame entre(li, axis=0, ignore_index=True) dos textos considerados positivos.
frame = pd.concat(li, axis=0, ignore_index=True)

In [322]:
# Irá retornar as 5 primeiras linhas do frame.
frame.head()

,0,1,2,3,4,5,6,7,8
0,Bromwell High is a cartoon comedy. It ran at t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Homelessness (or Houselessness as George Carli...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brilliant over-acting by Lesley Ann Warren. Be...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,This is easily the most underrated film inn th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,This is not the typical Mel Brooks film. It wa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [323]:
# Retorna a quantidade de linhas e colunas do DataFrame
frame.shape

(12500, 9)

In [324]:
# Retorna um objeto booleano do mesmo tamanho, indicando se os valores não são NA. 
pd.notna(frame[1])

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
12470    False
12471    False
12472    False
12473    False
12474    False
12475    False
12476    False
12477    False
12478    False
12479    False
12480    False
12481    False
12482    False
12483    False
12484    False
12485    False
12486    False
12487    False
12488    False
12489    False
12490    False
12491    False
12492    False
12493    False
12494    False
12495    False
12496    False
12497    False
12498    False
12499    False
Name: 1, Length: 12500, dtype: bool

In [325]:
frame[pd.notna(frame[1])]

,0,1,2,3,4,5,6,7,8
5262,Wrestlemania 14 is not often looked as one of ...,C,Vs Aguila.<br /><br />Taka gets a surprising ...,C,"Vs Owen Hart Stipulation here, is Chyna is ha...",C,". Before I review the match, I'd like to note ...",C,Vs Steve Austin. Big heat for Tyson. Crowd go...
10321,As Most Off You Might off Seen Star Wars: Retu...,might have a party At The end And They Just Pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [326]:
# Tratando os dados(positivos), preenchendo os valores nulos(NaN) e exibindo.
df_positivo = frame.fillna("")
df_positivo["frase"] = df_positivo[0] + df_positivo[1] + df_positivo[2] + df_positivo[3] + df_positivo[4] + df_positivo[5] + df_positivo[6] + df_positivo[7] + df_positivo[8]
df_positivo.head()

,0,1,2,3,4,5,6,7,8,frase
0,Bromwell High is a cartoon comedy. It ran at t...,,,,,,,,,Bromwell High is a cartoon comedy. It ran at t...
1,Homelessness (or Houselessness as George Carli...,,,,,,,,,Homelessness (or Houselessness as George Carli...
2,Brilliant over-acting by Lesley Ann Warren. Be...,,,,,,,,,Brilliant over-acting by Lesley Ann Warren. Be...
3,This is easily the most underrated film inn th...,,,,,,,,,This is easily the most underrated film inn th...
4,This is not the typical Mel Brooks film. It wa...,,,,,,,,,This is not the typical Mel Brooks film. It wa...


In [327]:
# Indica que a resposta para o resultado(binário) positivo será = 1
df_positivo["resp"] = 1

In [328]:
df_positivo = df_positivo[["frase", "resp"]]

In [329]:
# use o caminho onde a pasta "aclImdb" estiver no seu computar ex: ...\aclImdb\train\neg
path = r'C:\Users\alison.americo.lira\Documents\classificador_de_filmes\aclImdb\train\neg' 

all_files = glob.glob(path + "/*.txt")
li = []
for filename in all_files:
    df = pd.read_table(filename, index_col=None, header=None, sep="|")
    li.append(df)

In [330]:
# Irá fazer o merge no frame entre(li, axis=0, ignore_index=True) dos textos considerados negativos.
frame = pd.concat(li, axis=0, ignore_index=True)

In [331]:
# Irá retornar as 5 primeiras linhas do frame.
frame.head()

,0,1,2,3,4,5,6
0,Story of a man who has unnatural feelings for ...,NaN,NaN,NaN,NaN,NaN,NaN
1,Airport '77 starts as a brand new luxury 747 p...,NaN,NaN,NaN,NaN,NaN,NaN
2,This film lacked something I couldn't put my f...,NaN,NaN,NaN,NaN,NaN,NaN
3,"Sorry everyone,,, I know this is supposed to b...",NaN,NaN,NaN,NaN,NaN,NaN
4,When I was little my parents took me along to ...,NaN,NaN,NaN,NaN,NaN,NaN


In [332]:
# Retorna a quantidade de linhas e colunas do DataFrame()
frame.shape

(12500, 7)

In [333]:
pd.notna(frame[1])

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
12470    False
12471    False
12472    False
12473    False
12474    False
12475    False
12476    False
12477    False
12478    False
12479    False
12480    False
12481    False
12482    False
12483    False
12484    False
12485    False
12486    False
12487    False
12488    False
12489    False
12490    False
12491    False
12492    False
12493    False
12494    False
12495    False
12496    False
12497    False
12498    False
12499    False
Name: 1, Length: 12500, dtype: bool

In [334]:
# Tratando os dados(negativos), preenchendo os valores nulos(NaN) e exibindo.
df_negativo = frame.fillna("")
df_negativo["frase"] = df_negativo[0] + df_negativo[1] + df_negativo[2] + df_negativo[3] + df_negativo[4] + df_negativo[5] + df_negativo[6]
df_negativo.head()

,0,1,2,3,4,5,6,frase
0,Story of a man who has unnatural feelings for ...,,,,,,,Story of a man who has unnatural feelings for ...
1,Airport '77 starts as a brand new luxury 747 p...,,,,,,,Airport '77 starts as a brand new luxury 747 p...
2,This film lacked something I couldn't put my f...,,,,,,,This film lacked something I couldn't put my f...
3,"Sorry everyone,,, I know this is supposed to b...",,,,,,,"Sorry everyone,,, I know this is supposed to b..."
4,When I was little my parents took me along to ...,,,,,,,When I was little my parents took me along to ...


In [335]:
# Indica que a resposta para o resultado(binário) negativo será = 0
df_negativo["resp"] = 0

In [336]:
df_negativo = df_negativo[["frase", "resp"]]

In [337]:
# Irá juntas as duas bases de arquivos negativos e positivos.
mod = pd.concat([df_positivo, df_negativo] , axis=0, ignore_index= True)

In [338]:
# Retorna a quantidade de linhas e colunas do mod
mod.shape

(25000, 2)

In [339]:
# Irá limpar o texto
# Remove a pontuação e irá deixar o texto padronizado, as frases tudo maiúscula ou minúscula
mod["frase_tratada"]=mod.frase.apply(lambda x : x.lower())

In [340]:
mod["frase_tratada"]=mod.frase_tratada.apply(lambda x: unicodedata.normalize('NFD', x).encode('ascii', 'ignore').decode("utf-8"))

In [341]:
mod["frase_tratada"] = mod.frase_tratada.apply(lambda x : re.sub(r"[^a-z0-9\s]", "",x))

In [342]:
# Carrega os dados de treino
reviews_train = load_files("aclImdb/train/")

In [343]:
text_train, y_train = reviews_train.data, reviews_train.target

In [344]:
# Exibe o número de documentos dentro da pasta "train"
print("Número de documentos nos dados(train): {}".format(len(text_train)))

Número de documentos nos dados(train): 25000


In [345]:
print("Amostras por classe(train): {}".format(np.bincount(y_train)))

Amostras por classe(train): [12500 12500]


In [346]:
# Carrega os dados de teste
reviews_test = load_files("aclImdb/test/")

In [347]:
text_test, y_test = reviews_test.data, reviews_test.target

In [348]:
# Exibe o número de documentos dentro da pasta "test"
print("Número de documentos em dados de teste: {}".format(len(text_test)))

Número de documentos em dados de teste: 25000


In [349]:
print("Amostras por classe (teste): {}".format(np.bincount(y_test)))

Amostras por classe (teste): [12500 12500]


In [350]:
# min_df (= 5): define a frequência mínima de uma palavra para ser contada como um recurso
# ngram_range (= (2,2)): O parâmetro ngram_range é uma tupla. Define o comprimento mínimo e máximo da sequência de tokens considerados. 
vectorizer = CountVectorizer(min_df=5, ngram_range=(2, 2))

In [351]:
X_train = vectorizer.fit(text_train).transform(text_train)

In [352]:
X_test = vectorizer.transform(text_test)

In [353]:
print("Tamanho do vocabulário: {}".format(len(vectorizer.vocabulary_)))

Tamanho do vocabulário: 129549


In [354]:
print("X_train:\n{}".format(repr(X_train)))

X_train:
<25000x129549 sparse matrix of type '<class 'numpy.int64'>'
	with 3607330 stored elements in Compressed Sparse Row format>


In [355]:
print("X_test: \n{}".format(repr(X_test)))

X_test: 
<25000x129549 sparse matrix of type '<class 'numpy.int64'>'
	with 3392376 stored elements in Compressed Sparse Row format>


In [356]:
feature_names = vectorizer.get_feature_names()

In [357]:
print("Número de características: {}".format(len(feature_names)))

Número de características: 129549


In [358]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [359]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)

In [360]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [361]:
print("Melhor pontuação de validação cruzada: {:.2f}".format(grid.best_score_))

Melhor pontuação de validação cruzada: 0.88


In [362]:
print("Melhores parâmetros: ", grid.best_params_)

Melhores parâmetros:  {'C': 1}


In [363]:
print("Melhor estimador: ", grid.best_estimator_)

Melhor estimador:  LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [364]:
lr = grid.best_estimator_

In [365]:
lr.fit(X_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [366]:
lr.predict(X_test)

array([1, 0, 1, ..., 0, 0, 0])

In [367]:
print("Pontuação: {:.2f}".format(lr.score(X_test, y_test)))

Pontuação: 0.88


In [ ]:
# verificar como o modelo se comporta em dados individuais: Positivos e Negativos.

In [368]:
pos = ["I've seen this story before but my kids haven't. Boy with troubled past joins military, faces his past, falls in love and becomes a man. "
       "The mentor this time is played perfectly by Kevin Costner; An ordinary man with common everyday problems who lives an extraordinary "
       "conviction, to save lives. After losing his team he takes a teaching position training the next generation of heroes. The young troubled "
       "recruit is played by Kutcher. While his scenes with the local love interest are a tad stiff and don't generate enough heat to melt butter, "
       "he compliments Costner well. I never really understood Sela Ward as the neglected wife and felt she should of wanted Costner to quit out of "
       "concern for his safety as opposed to her selfish needs. But her presence on screen is a pleasure. The two unaccredited stars of this movie "
       "are the Coast Guard and the Sea. Both powerful forces which should not be taken for granted in real life or this movie. The movie has some "
       "slow spots and could have used the wasted 15 minutes to strengthen the character relationships. But it still works. The rescue scenes are "
       "intense and well filmed and edited to provide maximum impact. This movie earns the audience applause. And the applause of my two sons."]
print("Pos prediction: {}". format(lr.predict(vectorizer.transform(pos))))

Pos prediction: [1]


In [369]:
neg = ["David Bryce\'s comments nearby are exceptionally well written and informative as almost say everything "
       "I feel about DARLING LILI. This massive musical is so peculiar and over blown, over produced and must have "
       "caused ruptures at Paramount in 1970. It cost 22 million dollars! That is simply irresponsible. DARLING LILI "
       "must have been greenlit from a board meeting that said \"hey we got that Pink Panther guy and that Sound Of Music gal... "
       "lets get this too\" and handed over a blank cheque. The result is a hybrid of GIGI, ZEPPELIN, HALF A SIXPENCE, some MGM 40s "
       "song and dance numbers of a style (daisies and boaters!) so hopelessly old fashioned as to be like musical porridge, and MATA HARI "
       "dramatics. The production is colossal, lush, breathtaking to view, but the rest: the ridiculous romance, Julie looking befuddled, Hudson "
       "already dead, the mistimed comedy, and the astoundingly boring songs deaden this spectacular film into being irritating. LILI is"
       " like a twee 1940s mega musical with some vulgar bits to spice it up. STAR! released the year before sadly crashed and now is being "
       "finally appreciated for the excellent film is genuinely is... and Andrews looks sublime, mature, especially in the last half hour......"
       "but LILI is POPPINS and DOLLY frilly and I believe really killed off the mega musical binge of the 60s..... "
       "and made Andrews look like Poppins again... which I believe was not Edwards intention. Paramount must have collectively fainted "
       "when they saw this: and with another $20 million festering in CATCH 22, and $12 million in ON A CLEAR DAY and $25 million in PAINT YOUR WAGON...."
       "they had a financial abyss of CLEOPATRA proportions with $77 million tied into 4 films with very uncertain futures. Maybe they should have asked seer "
       "Daisy Gamble from ON A CLEAR DAY ......LILI was very popular on immediate first release in Australia and ran in 70mm cinemas for months but it failed "
       "once out in the subs and the sticks and only ever surfaced after that on one night stands with ON A CLEAR DAY as a Sunday night double. Thank "
       "god Paramount had their simple $1million (yes, ONE MILLION DOLLAR) film LOVE STORY and that $4 million dollar gangster pic THE GODFATHER "
       "also ready to recover all the $77 million in just the next two years....for just $5m.... incredible!"]
print("Neg prediction: {}". format(lr.predict(vectorizer.transform(neg))))

Neg prediction: [0]
